> **Public redacted copy** — outputs preserved, code shown with minimal changes.
> A few lines that perform data access or model training have been commented with `[redacted]` notes.


In [ ]:
# ---- Figure autosave helper (run once per session) ----
import os
import matplotlib.pyplot as plt

os.makedirs("figures", exist_ok=True)
_counter = {"n": 0}
_old_show = plt.show

def _save_then_show(*args, **kwargs):
    _counter["n"] += 1
    fp = f"figures/fig_{_counter['n']:02d}.png"
    plt.gcf().savefig(fp, dpi=300, bbox_inches="tight")
    _old_show(*args, **kwargs)

plt.show = _save_then_show

from matplotlib.figure import Figure
_old_fig_show = Figure.show
def _fig_save_then_show(self, *args, **kwargs):
    _counter["n"] += 1
    fp = f"figures/fig_{_counter['n']:02d}.png"
    self.savefig(fp, dpi=300, bbox_inches="tight")
    _old_fig_show(self, *args, **kwargs)
Figure.show = _fig_save_then_show

print("Autosave-on-show enabled → .[…redacted-path…]")



# Paper 3 — Supervised Logistic Model with Macro Variables

This notebook extends the Paper 2 setup by adding macro-financial variables (inflation, real rate, QE proxy) to predict a **12‑month‑ahead recession indicator**.  
It also produces **live forecasts** after the label cut-off (Mar 2024).

**Inputs expected:**
- Main macro dataset (CSV): `macro_core_monthly_1962-present_v03.csv` (or update the path below)
  - Columns required: `Date, CPI_YoY, CPI_YoY_lag1, Real10y_spliced, FedBal_YoY, YC_10y_3m`
- Recession indicator (monthly, 0/1) in one of two ways:
  1) Included in the macro CSV as column **`recession`** (same as Paper 2), **or**
  2) Provided in a separate CSV: `nber_recession_monthly.csv` with columns `Date, recession`

**Conventions**
- Date is monthly period end or first-of-month; parsed as datetime.
- Label is created as `recession_t12 = recession.shift(-12)`.
- Training window ends **2024-03**; forecasts start **2024-04**.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from pathlib import Path


In [ ]:
# --- Paths & existence checks ---
DATA_DIR = Path("..[…redacted-path…]")
MACRO_CSV = DATA_DIR / "macro_core_monthly_1962-present_v03.csv"
NBER_CSV  = DATA_DIR / "usrec_monthly.csv"

missing = [p.name for p in [MACRO_CSV, NBER_CSV] if not p.exists()]
if missing:
    raise FileNotFoundError(
        f"Missing expected file(s) in ..[…redacted-path…] {', '.join(missing)}. "
        "Check data folder mapping or filenames."
    )


In [ ]:
# --- Load macro CSV robustly (handles various date col names) ---
#raw = pd.read_csv(MACRO_CSV)
# [redacted] Critical operation disabled in public copy (data access or training).
raw.columns = [c.strip() for c in raw.columns]

date_col_candidates = ["Date", "DATE", "date", "observation_date", "Month", "month"]
date_col = next((c for c in date_col_candidates if c in raw.columns), None)
if date_col is None:
    date_col = raw.columns[0]

raw = raw.rename(columns={date_col: "Date"})
raw["Date"] = pd.to_datetime(raw["Date"], errors="coerce")

macro = (raw.dropna(subset=["Date"])
             .sort_values("Date")
             .reset_index(drop=True))

for c in macro.columns:
    if c != "Date":
        macro[c] = pd.to_numeric(macro[c], errors="coerce")

print("Macro loaded:", macro.shape, "|", macro["Date"].min().date(), "→", macro["Date"].max().date())


Macro loaded: (766, 6) | 1962-01-01 → 2025-10-01


In [ ]:
# --- Load NBER monthly recession file and standardize to ['Date','recession'] ---
#nber = pd.read_csv(NBER_CSV)
# [redacted] Critical operation disabled in public copy (data access or training).

if "Date" not in nber.columns:
    if "DATE" in nber.columns:
        nber = nber.rename(columns={"DATE": "Date"})
    else:
        nber = nber.rename(columns={nber.columns[0]: "Date"})
nber["Date"] = pd.to_datetime(nber["Date"], errors="coerce")

label_col = next((c for c in ["recession", "USREC", "USRECM"] if c in nber.columns), None)
if label_col is None:
    raise KeyError("Could not find a recession indicator column in NBER CSV (expected one of recession/USREC/USRECM).")

nber["recession"] = pd.to_numeric(nber[label_col], errors="coerce").fillna(0).astype(int)
nber = nber[["Date", "recession"]].sort_values("Date")

df = (macro.merge(nber, on="Date", how="left")
           .sort_values("Date")
           .reset_index(drop=True))



In [ ]:
# --- Quick diagnostics (safe to delete ONCE CONFIRMED) ---
print("Merged df:", df.shape)
print("Columns:", df.columns.tolist())
print("Date coverage:", df["Date"].min().date(), "→", df["Date"].max().date())
df.head(3)


Merged df: (766, 7)
Columns: ['Date', 'CPI_YoY', 'CPI_YoY_lag1', 'Real10y_spliced', 'FedBal_YoY', 'YC_10y_3m', 'recession']
Date coverage: 1962-01-01 → 2025-10-01


Date  CPI_YoY  CPI_YoY_lag1  Real10y_spliced  FedBal_YoY  YC_10y_3m  \
0 1962-01-01  0.67024           NaN          1.69976         NaN       1.36   
1 1962-02-01  0.90483       0.67024          1.94517         NaN       1.31   
2 1962-03-01  1.10590       0.90483          1.67410         NaN       1.21   

   recession  
0        0.0  
1        0.0  
2        0.0

In [ ]:
# --- Feature engineering, windows, and splits (clean) ---
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, brier_score_loss, roc_curve

# Ensure Date is the index (idempotent)
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date").set_index("Date")
else:
    # already indexed by Date
    if df.index.name != "Date":
        raise KeyError(f"Expected index named 'Date', got {df.index.name!r}")
    df = df.sort_index()

# 12-month-ahead target
df["recession"] = pd.to_numeric(df["recession"], errors="coerce").fillna(0).astype(int)
df["recession_t12"] = df["recession"].shift(-12)

# Windows (spec)
TRAIN_START = pd.Timestamp("1962-01-01")
TRAIN_END   = pd.Timestamp("2024-03-01")
HOLDOUT_START = pd.Timestamp("2017-01-01")

FEATURES = ["CPI_YoY_lag1","Real10y_spliced","FedBal_YoY","YC_10y_3m"]

# Early, friendly check (recommended)
missing = [c for c in FEATURES if c not in df.columns]
if missing:
    raise KeyError(f"Missing required FEATURES in df: {missing}")

# Masks
train_mask     = (df.index >= TRAIN_START) & (df.index <= TRAIN_END)
test_mask      = (df.index >= HOLDOUT_START) & (df.index <= TRAIN_END)
forecast_mask  = (df.index >  TRAIN_END)

# Splits
train_df  = df.loc[train_mask].dropna(subset=["recession_t12"]).copy()
test_df   = df.loc[test_mask].copy()
future_df = df.loc[forecast_mask].copy()

print("Train:", train_df.index.min().date(), "→", train_df.index.max().date(), "| n =", len(train_df))
print("Test: ", test_df.index.min().date(),  "→", test_df.index.max().date(),  "| n =", len(test_df))
print("Forecast window starts:", future_df.index.min().date() if len(future_df) else "—")


Train: 1962-01-01 → 2024-03-01 | n = 747
Test:  2017-01-01 → 2024-03-01 | n = 87
Forecast window starts: 2024-04-01


In [ ]:
# --- Model 1: Simple logit (statsmodels) fit ---
X_train = sm.add_constant(train_df[FEATURES], has_constant="add").dropna()
y_train = train_df.loc[X_train.index, "recession_t12"].astype(int)

logit = sm.Logit(y_train, X_train)
#res   = logit.fit(disp=False)
# [redacted] Critical operation disabled in public copy (data access or training).


In [ ]:
# --- Model 1: Evaluation on 2017+ holdout ---
X_test = sm.add_constant(test_df[FEATURES], has_constant="add").dropna()
y_test = test_df.loc[X_test.index, "recession_t12"].astype(int)

p_hat  = res.predict(X_test)

auc   = roc_auc_score(y_test, p_hat)
brier = brier_score_loss(y_test, p_hat)
print(f"Model 1 (Simple Logit) — AUC: {auc:.3f} | Brier: {brier:.3f}")

# ROC curve
fpr, tpr, _ = roc_curve(y_test, p_hat)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f"AUC={auc:.3f}")
plt.plot([0,1],[0,1],'--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC — Holdout (2017+)")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

# (Appendix material—move these out of the main flow if you keep them)
# from sklearn.metrics import average_precision_score
# auprc = average_precision_score(y_test, p_hat); print(f"AUPRC: {auprc:.3f}")
# print(res.summary())
# # Time-series prob plot also belongs in appendix



Model 1 (Simple Logit) — AUC: 0.694 | Brier: 0.033


In [ ]:
# --- Model 2: Regularized logit (L2, sklearn) ---
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

# Train data
Xtr_lin = train_df[FEATURES]
ytr     = train_df["recession_t12"].astype(int)

# Test data
Xte_lin = test_df[FEATURES]
yte     = test_df["recession_t12"].astype(int)

pipe_l2 = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  StandardScaler()),
    ("clf",    LogisticRegression(penalty="l2", solver="lbfgs", max_iter=1000))
])

#pipe_l2.fit(Xtr_lin, ytr)
# [redacted] Critical operation disabled in public copy (data access or training).
p_hat_reg = pipe_l2.predict_proba(Xte_lin)[:, 1]

auc_reg   = roc_auc_score(yte, p_hat_reg)
brier_reg = brier_score_loss(yte, p_hat_reg)
print(f"Model 2 (L2 logit) — AUC: {auc_reg:.3f} | Brier: {brier_reg:.3f}")


Model 2 (L2 logit) — AUC: 0.676 | Brier: 0.055


In [ ]:
# --- Model 3: Nonlinear L2 (RBF features on YC_10y_3m) ---
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

# Build RBFs from TRAIN quantiles of the yield-curve spread
yc_train = train_df["YC_10y_3m"].astype(float)
q10, q50, q90 = np.quantile(yc_train.dropna(), [0.10, 0.50, 0.90])

# bandwidth = average gap between adjacent knots (simple, stable)
sigma = np.mean([abs(q50 - q10), abs(q90 - q50)]) or (yc_train.std() or 1.0)

def rbf(x, c, s):
    z = (x - c) / (s if s != 0 else 1.0)
    return np.exp(-0.5 * z * z)

# Create RBF features on train/test
for frame in (train_df, test_df):
    x = frame["YC_10y_3m"].astype(float)
    frame["YC_rbf1"] = rbf(x, q10, sigma)
    frame["YC_rbf2"] = rbf(x, q50, sigma)
    frame["YC_rbf3"] = rbf(x, q90, sigma)

FEATURES_NL = ["CPI_YoY_lag1","Real10y_spliced","FedBal_YoY","YC_rbf1","YC_rbf2","YC_rbf3"]

Xtr_nl = train_df[FEATURES_NL]
Xte_nl = test_df[FEATURES_NL]
ytr    = train_df["recession_t12"].astype(int)
yte    = test_df["recession_t12"].astype(int)

pipe_nl = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  StandardScaler()),
    ("clf",    LogisticRegression(penalty="l2", solver="lbfgs", max_iter=1000))
])

#pipe_nl.fit(Xtr_nl, ytr)
# [redacted] Critical operation disabled in public copy (data access or training).
p_hat_nl = pipe_nl.predict_proba(Xte_nl)[:, 1]

auc_nl   = roc_auc_score(yte, p_hat_nl)
brier_nl = brier_score_loss(yte, p_hat_nl)
print(f"Model 3 (Nonlinear YC + L2) — AUC: {auc_nl:.3f} | Brier: {brier_nl:.3f}")


Model 3 (Nonlinear YC + L2) — AUC: 0.688 | Brier: 0.040


In [ ]:
# --- Side-by-side ROC comparison ---
from sklearn.metrics import roc_curve

fpr1, tpr1, _ = roc_curve(y_test, p_hat)       # Model 1 (statsmodels)
fpr2, tpr2, _ = roc_curve(yte, p_hat_reg)      # Model 2 (L2)
fpr3, tpr3, _ = roc_curve(yte, p_hat_nl)       # Model 3 (Nonlinear L2)

plt.figure(figsize=(7,5))
plt.plot(fpr1, tpr1, lw=2, label=f"Simple Logit (AUC={auc:.3f})")
plt.plot(fpr2, tpr2, lw=2, label=f"L2 Logit (AUC={auc_reg:.3f})")
plt.plot(fpr3, tpr3, lw=2, label=f"Nonlinear YC + L2 (AUC={auc_nl:.3f})")
plt.plot([0,1],[0,1],'--', lw=1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC — 2017+ holdout")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
# --- Final model selection already printed; keep variables: best_name, auc, brier, etc. ---

def predict_probs_final(df_features):
    """
    Input: DataFrame with at least the base FEATURES columns.
           For nonlinear winner, YC RBFs are computed using TRAIN knots (q10,q50,q90,sigma).
    Output: 1D numpy array of probabilities (12m-ahead recession).
    """
    if best_name == "simple_logit":
        X = sm.add_constant(df_features[FEATURES], has_constant="add")
        # align to training design (important if columns get re-ordered)
        X = X.reindex(columns=logit.exog_names, fill_value=0).dropna()
        return res.predict(X).values

    elif best_name == "l2_logit":
        return pipe_l2.predict_proba(df_features[FEATURES])[:, 1]

    elif best_name == "nonlinear_l2":
        df2 = df_features.copy()
        x = df2["YC_10y_3m"].astype(float)
        s = (sigma if sigma != 0 else 1.0)
        df2["YC_rbf1"] = np.exp(-0.5 * ((x - q10)[…redacted-path…]
        df2["YC_rbf2"] = np.exp(-0.5 * ((x - q50)[…redacted-path…]
        df2["YC_rbf3"] = np.exp(-0.5 * ((x - q90)[…redacted-path…]
        return pipe_nl.predict_proba(df2[FEATURES_NL])[:, 1]

    else:
        raise ValueError("Unknown final model selection.")


In [ ]:
# --- Minimal shim for Cell [13] ---
# Ensures best_name, logit, res exist after a restart, without touching other cells.
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 1) Winner flag (your flow uses the simple logit)
if 'best_name' not in globals():
    best_name = 'simple_logit'

# 2) Recreate fitted model ONLY if missing (uses existing train_df + FEATURES from Cell [6])
need_refit = ('logit' not in globals()) or ('res' not in globals())
if need_refit:
    # Sanity checks
    if 'train_df' not in globals() or 'FEATURES' not in globals():
        raise RuntimeError("Missing train_df and/or FEATURES. Re-run Cell [6] (feature engineering & splits) first.")
    missing = sorted(set(FEATURES + ['recession_t12']) - set(train_df.columns))
    if missing:
        raise RuntimeError(f"Training data missing required columns: {missing}. Re-run upstream prep cells.")

    X_train = sm.add_constant(train_df[FEATURES], has_constant='add').replace([np.inf, -np.inf], np.nan).dropna()
    y_train = train_df.loc[X_train.index, 'recession_t12'].astype(int)
    logit = sm.Logit(y_train, X_train)
#    res   = logit.fit(disp=False)
# [redacted] Critical operation disabled in public copy (data access or training).
    print("Refit simple_logit (logit/res) for forecasting.")


In [ ]:
# --- Live forecasts for post-TRAIN_END observations (winner only) ---
# Uses the features already observed in future_df (post 2024-03-01)
if len(future_df):
    p_live = predict_probs_final(future_df)
    live_forecasts = future_df.copy()
    live_forecasts["p_hat_final"] = p_live
    out_live = live_forecasts[["p_hat_final"]].copy()
    out_live.index.name = "Date"

    # Save
    out_live.to_csv("paper3_live_forecasts.csv")
    print("Saved: paper3_live_forecasts.csv  | rows:", len(out_live))
else:
    print("No rows in future_df — nothing to forecast yet.")


Saved: paper3_live_forecasts.csv  | rows: 19


In [ ]:
# --- Shim for scenarios: define future_index and H, and sanity-check inputs ---
import numpy as np
import pandas as pd

# 1) future_index and H come from your already-built future_df (post-TRAIN_END)
if 'future_df' not in globals():
    raise RuntimeError("future_df is not defined. Re-run the split cell that creates future_df.")
future_index = future_df.index
H = len(future_index)
if H == 0:
    raise RuntimeError("future_df has zero rows (no post-TRAIN_END dates). Nothing to scenario-forecast.")

# 2) Ensure FEATURES are available and covered by steps/bounds
if 'FEATURES' not in globals():
    raise RuntimeError("FEATURES not defined. Re-run the cell that sets FEATURES.")
missing_steps  = sorted(set(FEATURES) - set(steps.keys()))
missing_bounds = sorted(set(FEATURES) - set(bounds.keys()))
if missing_steps or missing_bounds:
    raise RuntimeError(
        f"steps/bounds missing keys.\n"
        f"  Missing in steps:  {missing_steps}\n"
        f"  Missing in bounds: {missing_bounds}\n"
        f"Add entries for all FEATURES or drop those features from the scenario."
    )

# 3) Optional: verify last row has no NaNs for these FEATURES
if 'df' not in globals():
    raise RuntimeError("df not defined. Re-run earlier loading/feature cells.")
_ = df.iloc[-1][FEATURES].astype(float)
if _.isna().any():
    bad = list(_.index[_.isna()])
    raise RuntimeError(f"Latest row has NaNs for: {bad}. Fix upstream feature engineering before scenarios.")


In [ ]:
# --- Multi-feature scenario setup (replace the simpler YC-only bit) ---

# last observed levels for all FEATURES
last = df.iloc[-1]
last_vals = last[FEATURES].astype(float)

def path_linear(start, step, n, lo=None, hi=None):
    arr = start + step * np.arange(1, n+1, dtype=float)
    if lo is not None: arr = np.maximum(arr, lo)
    if hi is not None: arr = np.minimum(arr, hi)
    return arr

# monthly step sizes (percentage-point moves per month)
steps = {
    "CPI_YoY_lag1":    {"baseline": -0.05, "easing": -0.15, "stress": +0.10},
    "Real10y_spliced": {"baseline":  0.00, "easing": -0.10, "stress": +0.08},
    "YC_10y_3m":       {"baseline": +0.02, "easing": +0.08, "stress": -0.06},
    "FedBal_YoY":      {"baseline":  0.00, "easing": +1.50, "stress": -1.50},
}
bounds = {
    "CPI_YoY_lag1":    (-1.0, 8.0),
    "Real10y_spliced": (-2.0, 5.0),
    "YC_10y_3m":       (-3.0, 3.0),
    "FedBal_YoY":      (-50.0, 100.0),
}

def make_scenarios(last_vals, n, features):
    scens = {}
    for name in ["baseline", "easing", "stress"]:
        df_s = pd.DataFrame(index=future_index, columns=features, dtype=float)
        for feat in features:
            df_s[feat] = path_linear(
                start=last_vals[feat],
                step=steps[feat][name],
                n=n,
                lo=bounds[feat][0], hi=bounds[feat][1]
            )
        scens[name] = df_s
    return scens

scenarios = make_scenarios(last_vals, H, FEATURES)


In [ ]:
# --- Shim for MC bands: ensure prerequisites exist ---
import numpy as np, pandas as pd

# future horizon / index
if 'future_df' not in globals():
    raise RuntimeError("future_df is not defined. Re-run the split cell that creates train_df/test_df/future_df.")
future_index = future_df.index
H = len(future_index)
if H == 0:
    raise RuntimeError("future_df has zero rows (no post-TRAIN_END dates). Nothing to simulate.")

# features and last values
if 'FEATURES' not in globals():
    raise RuntimeError("FEATURES not defined. Re-run the cell that sets FEATURES.")
if 'df' not in globals():
    raise RuntimeError("df not defined. Re-run earlier loading/feature cells.")
last_vals = df.iloc[-1][FEATURES].astype(float)

# ensure make_scenarios & inputs exist
if 'make_scenarios' not in globals():
    raise RuntimeError("make_scenarios(...) is not defined. Re-run the scenario-setup cell that defines it.")
if 'steps' not in globals() or 'bounds' not in globals():
    raise RuntimeError("steps/bounds not defined. Re-run the scenario-setup cell.")
missing_steps  = sorted(set(FEATURES) - set(steps.keys()))
missing_bounds = sorted(set(FEATURES) - set(bounds.keys()))
if missing_steps or missing_bounds:
    raise RuntimeError(
        f"steps/bounds missing keys for FEATURES.\n"
        f"  Missing in steps:  {missing_steps}\n"
        f"  Missing in bounds: {missing_bounds}\n"
        f"Add entries or restrict FEATURES for scenarios."
    )

# (re)build scenarios if absent
if 'scenarios' not in globals():
    scenarios = make_scenarios(last_vals, H, FEATURES)

# predictor must exist
if 'predict_probs_final' not in globals():
    raise RuntimeError("predict_probs_final(...) not found. Re-run Cell [12] where it is defined.")


In [ ]:
# --- Monte Carlo input-uncertainty bands (vary YC_10y_3m only) ---
rng = np.random.default_rng(42)
yc_std = float(train_df["YC_10y_3m"].std())  # scale of typical monthly spread moves
N = 500  # number of MC paths

bands = {}
for name, df_s in scenarios.items():
    # replicate scenario YC path with Gaussian noise each month
    paths = []
    for _ in range(N):
        noisy = df_s.copy()
        noisy["YC_10y_3m"] = noisy["YC_10y_3m"].values + rng.normal(0.0, 0.5*yc_std, size=len(df_s))
        paths.append(predict_probs_final(noisy))
    arr = np.vstack(paths)  # N x H
    bands[name] = {
        "p50": np.median(arr, axis=0),
        "p10": np.percentile(arr, 10, axis=0),
        "p90": np.percentile(arr, 90, axis=0),
    }

# Collect to long and wide frames
band_frames = []
for name, d in bands.items():
    tmp = pd.DataFrame({"Date": future_index,
                        "p10": d["p10"], "p50": d["p50"], "p90": d["p90"]})
    tmp["scenario"] = name
    band_frames.append(tmp.set_index("Date"))
bands_long = pd.concat(band_frames).reset_index()
bands_wide = {}
for name in scenarios.keys():
    sub = bands_long[bands_long["scenario"] == name].set_index("Date")[["p10","p50","p90"]]
    bands_wide[name] = sub

# Save MC bands per scenario
for name, dfw in bands_wide.items():
    fn = f"paper3_scenario_probs_mc_{name}.csv"
    dfw.to_csv(fn)
    print("Saved:", fn, "| rows:", len(dfw))


Saved: paper3_scenario_probs_mc_baseline.csv | rows: 19
Saved: paper3_scenario_probs_mc_easing.csv | rows: 19
Saved: paper3_scenario_probs_mc_stress.csv | rows: 19


In [ ]:
# --- Scenario probabilities (winner only) + save a single canonical CSV ---
scenario_probs = {
    name: pd.Series(predict_probs_final(df_s), index=future_index, name=name)
    for name, df_s in scenarios.items()
}

scen_wide = pd.concat(scenario_probs.values(), axis=1)
scen_wide.index.name = "Date"
scen_wide.to_csv("paper3_scenario_probs.csv")
print("Saved: paper3_scenario_probs.csv | cols:", list(scen_wide.columns))


Saved: paper3_scenario_probs.csv | cols: ['baseline', 'easing', 'stress']


In [ ]:
# Appendix: PR vs threshold (final model)
from sklearn.metrics import precision_recall_curve

p_final_test = predict_probs_final(test_df)
prec, rec, thr = precision_recall_curve(y_test, p_final_test)

plt.figure(figsize=(6,4))
plt.plot(thr, prec[:-1], label="Precision")
plt.plot(thr, rec[:-1],  label="Recall")
# Optional: if you computed best_thr for the final model
# plt.axvline(x=best_thr, ls='--', label=f"Youden={best_thr:.3f}")
plt.xlabel("Threshold"); plt.ylabel("Score")
plt.title("Precision–Recall Tradeoff vs Threshold (Final Model)")
plt.legend(); plt.tight_layout(); plt.show()



In [ ]:
# --- Live forecasts (winner only) + save (replaces your cell [16]) ---
if len(future_df):
    p_live = predict_probs_final(future_df)
    live_forecasts = future_df.copy()
    live_forecasts["p_hat_final"] = p_live

    out_live = live_forecasts[["p_hat_final"]].copy()
    out_live.index.name = "Date"
    out_live.to_csv("paper3_live_forecasts.csv")
    print("Saved: paper3_live_forecasts.csv  | rows:", len(out_live))
else:
    print("No rows in future_df — nothing to forecast yet.")


Saved: paper3_live_forecasts.csv  | rows: 19


In [ ]:
# Appendix: quick live series plot (optional)
#out_live = pd.read_csv("paper3_live_forecasts.csv", parse_dates=["Date"]).set_index("Date")
# [redacted] Critical operation disabled in public copy (data access or training).
plt.figure(figsize=(10,4))
plt.plot(out_live.index, out_live["p_hat_final"])
plt.xlabel("Date"); plt.ylabel("Recession Prob (12m ahead)")
plt.title("Live Forecast — Apr 2024 onward (Final Model)")
plt.tight_layout(); plt.show()


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pathlib import Path

# ---------- 1) Historical monthly volatilities (on changes) ----------
# Use last 10 years for a realistic modern-vol estimate (tweak if you want)
hist_cut = df.index.max() - pd.DateOffset(years=10)
hist = df.loc[df.index >= hist_cut, FEATURES].astype(float).ffill()

# monthly change std per feature (percentage-point changes)
d_hist = hist.diff()
sigma = d_hist.std().to_dict()   # e.g., {"CPI_YoY_lag1": 0.25, ...}

# Reasonable hard bounds to avoid absurd values in sims
bounds = {
    "CPI_YoY_lag1": (-1.0, 8.0),
    "Real10y_spliced": (-2.0, 5.0),
    "YC_10y_3m": (-3.0, 3.0),
    "FedBal_YoY": (-50.0, 100.0),
}

# ---------- 2) Monte Carlo simulator around a deterministic scenario path ----------
def simulate_probs_around_path(path_df, sims=500, seed=42):
    """Return quantiles (p10, p50, p90) of recession_prob_t12 for each month."""
    rng = np.random.default_rng(seed)
    T = len(path_df)
    feats = path_df.columns.tolist()
    idx = path_df.index

    # Pre-allocate matrix for probabilities: (sims, T)
    P = np.empty((sims, T), dtype=float)

    for s in range(sims):
        # Start at deterministic path; add Gaussian monthly shocks to changes
        sim = path_df.copy()
        for f in feats:
            # random monthly shocks to the LEVEL via shocks on changes (random walk with drift)
            shocks = rng.normal(loc=0.0, scale=sigma.get(f, 0.0), size=T)
            sim[f] = sim[f].iloc[0] + np.cumsum(np.r_[0.0, np.diff(path_df[f].values) + shocks[1:]])
            # clamp
            sim[f] = sim[f].clip(lower=bounds[f][0], upper=bounds[f][1])

        # Predict with fixed trained model
        p = pd.Series(predict_probs_final(sim), index=idx).astype(float)
        P[s, :] = p.values

    # Quantiles across simulations for each date
    q10 = np.quantile(P, 0.10, axis=0)
    q50 = np.quantile(P, 0.50, axis=0)
    q90 = np.quantile(P, 0.90, axis=0)

    out = pd.DataFrame({"Date": idx, "p10": q10, "p50": q50, "p90": q90}).set_index("Date")
    return out


In [ ]:
# Ensure you already created `scenarios` (baseline/easing/stress) as before
# stable seeds per scenario
seed_map = {"baseline": 123, "easing": 124, "stress": 125}

mc = {}
for name, path_df in scenarios.items():
    mc[name] = simulate_probs_around_path(
        path_df[FEATURES], sims=500, seed=seed_map[name]
    )

# Plot ribbons + medians
plt.figure(figsize=(11,5))
colors = {"baseline":"tab:blue","easing":"tab:green","stress":"tab:red"}

for name in ["baseline","easing","stress"]:
    q = mc[name]
    plt.fill_between(q.index, q["p10"], q["p90"], alpha=0.2, label=f"{name.capitalize()} (10–90%)",
                     color=colors[name])
    plt.plot(q.index, q["p50"], lw=2, color=colors[name], label=f"{name.capitalize()} median")

plt.ylim(0, 1)
plt.ylabel("Recession Prob (12m ahead)")
plt.title("Scenario-based Conditional Recession Probabilities — with Uncertainty Bands")
plt.legend(ncol=3, loc="upper center")
plt.tight_layout()
plt.show()

# Tidy save for appendix (long format)
out_list = []
for name, q in mc.items():
    tmp = q.reset_index().assign(scenario=name)
    out_list.append(tmp)
out = pd.concat(out_list, ignore_index=True)
out = out[["Date","scenario","p10","p50","p90"]]

out_path = Path("paper3_scenario_probs_mc.csv")  # root, canonical
out.to_csv(out_path, index=False)
print("Saved MC scenario bands →", out_path, "| rows:", len(out))


Saved MC scenario bands → paper3_scenario_probs_mc.csv | rows: 57


In [ ]:
# --- Scenario setup (robust to Date as column or unnamed index) ---
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 0) FEATURES (only if not already set)
try:
    FEATURES
except NameError:
    FEATURES = ["CPI_YoY_lag1", "Real10y_spliced", "FedBal_YoY", "YC_10y_3m"]

# 1) Ensure df exists and has a proper datetime index named "Date"
if "Date" in getattr(df, "columns", []):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date").set_index("Date")
else:
    # Already indexed by dates (possibly unnamed)
    df = df.copy()
    df.index = pd.to_datetime(df.index, errors="coerce")
    # Can't use dropna(subset=[df.index.name]) if name is None; use index mask instead
    df = df.loc[~pd.isna(df.index)].sort_index()
    if df.index.name != "Date":
        df.index.name = "Date"

# 2) Check that all FEATURES exist
missing_feats = sorted(set(FEATURES) - set(df.columns))
if missing_feats:
    raise KeyError(f"DataFrame is missing required FEATURES columns: {missing_feats}")

# 3) Last non-NA values for FEATURES and last observed date
last_row  = df[FEATURES].ffill().iloc[-1]
last_date = df.index.max()

# 4) Forecast horizon and future index
horizon_months = 24  # adjust to 36/48 if desired
future_index = pd.date_range(last_date + pd.offsets.MonthBegin(1),
                             periods=horizon_months, freq="MS")
H = len(future_index)

# 5) Values for the scenario generator
last_vals = last_row.astype(float)



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

# Use FEATURES from Cell E
Xtr = train_df[FEATURES].astype(float)
ytr = train_df["recession_t12"].astype(int)
Xte = test_df[FEATURES].astype(float)
yte = test_df["recession_t12"].astype(int)

pipe_l2 = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),   # fit on TRAIN only
    ("scale",  StandardScaler()),
    ("clf",    LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=1000))
])
#pipe_l2.fit(Xtr, ytr)
# [redacted] Critical operation disabled in public copy (data access or training).

p_hat_reg = pipe_l2.predict_proba(Xte)[:, 1]
auc_reg   = roc_auc_score(yte, p_hat_reg)
brier_reg = brier_score_loss(yte, p_hat_reg)
print(f"Model 2 (L2 logit) — AUC: {auc_reg:.3f} | Brier: {brier_reg:.3f}")


Model 2 (L2 logit) — AUC: 0.676 | Brier: 0.055


In [ ]:
# --- Model 3: Nonlinear L2 (RBF features on YC_10y_3m) ---
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

# RBF knots from TRAIN quantiles of YC_10y_3m
yc_train = train_df["YC_10y_3m"].astype(float).dropna()
q10, q50, q90 = np.quantile(yc_train, [0.10, 0.50, 0.90])
sigma = np.mean([abs(q50 - q10), abs(q90 - q50)]) or (yc_train.std() or 1.0)

def rbf(series, c, s):
    z = (series - c) / (s if s != 0 else 1.0)
    return np.exp(-0.5 * z * z)

# Build temp frames with RBFs (don’t mutate originals)
train_nl = train_df.copy()
test_nl  = test_df.copy()
for frame in (train_nl, test_nl):
    x = frame["YC_10y_3m"].astype(float)
    frame["YC_rbf1"] = rbf(x, q10, sigma)
    frame["YC_rbf2"] = rbf(x, q50, sigma)
    frame["YC_rbf3"] = rbf(x, q90, sigma)

FEATURES_NL = ["CPI_YoY_lag1","Real10y_spliced","FedBal_YoY","YC_rbf1","YC_rbf2","YC_rbf3"]

Xtr_nl = train_nl[FEATURES_NL]
Xte_nl = test_nl[FEATURES_NL]
ytr_nl = train_nl["recession_t12"].astype(int)
yte_nl = test_nl["recession_t12"].astype(int)

pipe_nl = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  StandardScaler()),
    ("clf",    LogisticRegression(penalty="l2", solver="lbfgs", max_iter=1000))
])

#pipe_nl.fit(Xtr_nl, ytr_nl)
# [redacted] Critical operation disabled in public copy (data access or training).
p_hat_nl = pipe_nl.predict_proba(Xte_nl)[:, 1]

auc_nl   = roc_auc_score(yte_nl, p_hat_nl)
brier_nl = brier_score_loss(yte_nl, p_hat_nl)
print(f"Model 3 (Nonlinear YC + L2) — AUC: {auc_nl:.3f} | Brier: {brier_nl:.3f}")


Model 3 (Nonlinear YC + L2) — AUC: 0.688 | Brier: 0.040


In [ ]:
# --- ROC comparison on 2017+ holdout ---
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Model 1 used statsmodels and dropped NA rows; use the same y_test it produced
# (y_test and p_hat were created in your Model 1 cell)
fpr1, tpr1, _ = roc_curve(y_test, p_hat)

# Models 2 & 3 used imputer, so they score the full test_df rows
fpr2, tpr2, _ = roc_curve(yte,    p_hat_reg)
fpr3, tpr3, _ = roc_curve(yte_nl, p_hat_nl)

plt.figure(figsize=(7,5))
plt.plot(fpr1, tpr1, lw=2, label=f"Simple Logit (AUC={auc:.3f})")
plt.plot(fpr2, tpr2, lw=2, label=f"L2 Logit (AUC={auc_reg:.3f})")
plt.plot(fpr3, tpr3, lw=2, label=f"Nonlinear YC + L2 (AUC={auc_nl:.3f})")
plt.plot([0,1],[0,1],'--', lw=1)
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC — 2017+ holdout")
plt.legend(loc="lower right")
plt.tight_layout(); plt.show()


In [ ]:
#Ablation: Yield-Curve-Only Model

Xtr_min = train_df[["YC_10y_3m"]].astype(float)
Xte_min = test_df[["YC_10y_3m"]].astype(float)

sc_min  = StandardScaler()
#Xtrz_m  = sc_min.fit_transform(Xtr_min)
# [redacted] Critical operation disabled in public copy (data access or training).
Xtez_m  = sc_min.transform(Xte_min)

clf_min = LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=1000)
#clf_min.fit(Xtrz_m, ytr)
# [redacted] Critical operation disabled in public copy (data access or training).
p_hat_min = clf_min.predict_proba(Xtez_m)[:,1]


In [ ]:
# --- Model 3: Nonlinear L2 (RBF on YC_10y_3m) ---

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

# TRAIN-only knots and bandwidth
yc_train = train_df["YC_10y_3m"].astype(float).dropna().values
q10, q50, q90 = np.quantile(yc_train, [0.10, 0.50, 0.90])
sigma = np.mean([abs(q50 - q10), abs(q90 - q50)]) or (np.std(yc_train) or 1.0)

def rbf(x, c, s):
    z = (x - c) / (s if s != 0 else 1.0)
    return np.exp(-0.5 * z * z)

def featurize(df0):
    x = df0[["CPI_YoY_lag1","Real10y_spliced","FedBal_YoY","YC_10y_3m"]].astype(float).copy()
    x["YC_rbf1"] = rbf(x["YC_10y_3m"].values, q10, sigma)
    x["YC_rbf2"] = rbf(x["YC_10y_3m"].values, q50, sigma)
    x["YC_rbf3"] = rbf(x["YC_10y_3m"].values, q90, sigma)
    return x

Xtr_nl = featurize(train_df)
Xte_nl = featurize(test_df)
ytr_nl = train_df["recession_t12"].astype(int)
yte_nl = test_df["recession_t12"].astype(int)

pipe_nl = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  StandardScaler()),
    ("clf",    LogisticRegression(penalty="l2", solver="lbfgs", max_iter=1000))
])

#pipe_nl.fit(Xtr_nl, ytr_nl)
# [redacted] Critical operation disabled in public copy (data access or training).
p_hat_nl = pipe_nl.predict_proba(Xte_nl)[:, 1]

auc_nl   = roc_auc_score(yte_nl, p_hat_nl)
brier_nl = brier_score_loss(yte_nl, p_hat_nl)
print(f"Model 3 (Nonlinear YC + L2) — AUC: {auc_nl:.3f} | Brier: {brier_nl:.3f}")


Model 3 (Nonlinear YC + L2) — AUC: 0.688 | Brier: 0.040


In [ ]:
#appendix 

from sklearn.metrics import roc_curve, confusion_matrix, precision_score, recall_score, f1_score

fpr, tpr, thr = roc_curve(yte_nl, p_hat_nl)
best_thr_nl = thr[(tpr - fpr).argmax()]
y_pred_nl   = (p_hat_nl >= best_thr_nl).astype(int)

cm = confusion_matrix(yte_nl, y_pred_nl, labels=[0,1])
tn, fp, fn, tp = cm.ravel()
print(f"Youden thr (NL)={best_thr_nl:.3f} | TP={tp} FP={fp} FN={fn} TN={tn}")
print("Precision:", precision_score(yte_nl, y_pred_nl, zero_division=0),
      "Recall:",   recall_score(yte_nl, y_pred_nl, zero_division=0),
      "F1:",       f1_score(yte_nl, y_pred_nl, zero_division=0))


Youden thr (NL)=0.117 | TP=2 FP=27 FN=0 TN=58
Precision: 0.06896551724137931 Recall: 1.0 F1: 0.12903225806451613


In [ ]:
# assumes you still have p_hat (linear statsmodels), p_hat_reg (impute+L2), and p_hat_nl (nonlinear+L2)
from sklearn.metrics import roc_curve, roc_auc_score
plt.figure(figsize=(6,4))

def plot_roc(y, p, label):
    fpr, tpr, _ = roc_curve(y, p)
    auc = roc_auc_score(y, p)
    plt.plot(fpr, tpr, label=f"{label} (AUC={auc:.3f})")

plot_roc(y_test, p_hat,     "SM Logit (linear)")
plot_roc(y_test, p_hat_reg, "Sklearn L2 (linear)")
plot_roc(y_test, p_hat_nl,  "Sklearn L2 + nonlinear")
plt.plot([0,1],[0,1],'--', lw=1)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Holdout ROC (2017+)")
plt.legend(loc="lower right"); plt.tight_layout(); plt.show()



In [ ]:
# Appendix: threshold diagnostics helper (final model)
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

def evaluate_at_threshold(y_true, p, thr=0.5, title_suffix="(t=0.5)"):
    y_pred = (p >= thr).astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])  # rows=true, cols=pred
    tn, fp, fn, tp = cm.ravel()
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)

    print(f"Threshold {title_suffix}")
    print(f"TP={tp}  FP={fp}  FN={fn}  TN={tn}")
    print(f"Precision={prec:.3f}  Recall={rec:.3f}  F1={f1:.3f}")

    fig, ax = plt.subplots(figsize=(4,3))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(f"Confusion Matrix {title_suffix}")
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    ax.set_xticks([0,1]); ax.set_xticklabels(["No Recession","Recession"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["No Recession","Recession"])
    cm_norm = cm / cm.sum(axis=1, keepdims=True)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, f"{cm[i,j]}\n({cm_norm[i,j]:.2f})",
                    ha="center", va="center", fontsize=9)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout(); plt.show()


In [ ]:
# Appendix: evaluate a few thresholds for the final model on 2017+ holdout
from sklearn.metrics import roc_curve

# Final-model probabilities on the holdout
p_final_test = predict_probs_final(test_df)

# 0.50 threshold
evaluate_at_threshold(y_test, p_final_test, thr=0.50, title_suffix="(t=0.50)")

# Youden threshold
fpr_, tpr_, thr_ = roc_curve(y_test, p_final_test)
best_thr_final = thr_[(tpr_ - fpr_).argmax()]
print(f"Youden (final model) threshold: {best_thr_final:.3f}")
evaluate_at_threshold(y_test, p_final_test, thr=best_thr_final,
                      title_suffix=f"(t={best_thr_final:.3f}, Youden)")

# Base-rate threshold
thr_base_final = float(np.quantile(p_final_test, 1 - y_train.mean()))
evaluate_at_threshold(y_test, p_final_test, thr=thr_base_final,
                      title_suffix=f"(t={thr_base_final:.3f}, base-rate)")

# Early-warning 0.20
evaluate_at_threshold(y_test, p_final_test, thr=0.20,
                      title_suffix="(t=0.20, early-warning)")

# Target recall ≥ 0.50
def threshold_for_recall(y_true, p, target_recall=0.60):
    fpr, tpr, thr = roc_curve(y_true, p)
    i = np.argmax(tpr >= target_recall)
    return float(thr[i]) if i < len(thr) else 0.5

thr_recall = threshold_for_recall(y_test, p_final_test, target_recall=0.50)
evaluate_at_threshold(y_test, p_final_test, thr=thr_recall,
                      title_suffix=f"(t={thr_recall:.3f}, recall≥0.50)")


Threshold (t=0.50)
TP=0  FP=0  FN=2  TN=85
Precision=0.000  Recall=0.000  F1=0.000


Youden (final model) threshold: 0.047
Threshold (t=0.047, Youden)
TP=2  FP=28  FN=0  TN=57
Precision=0.067  Recall=1.000  F1=0.125


Threshold (t=0.237, base-rate)
TP=0  FP=8  FN=2  TN=77
Precision=0.000  Recall=0.000  F1=0.000


Threshold (t=0.20, early-warning)
TP=0  FP=8  FN=2  TN=77
Precision=0.000  Recall=0.000  F1=0.000


Threshold (t=0.053, recall≥0.50)
TP=1  FP=24  FN=1  TN=61
Precision=0.040  Recall=0.500  F1=0.074


In [ ]:
# Appendix: Parameter-uncertainty MC (only for simple_logit winner)
import numpy as np
import pandas as pd
import statsmodels.api as sm

if best_name == "simple_logit":
    beta_hat = res.params.values
    cov_hat  = res.cov_params().values
    design_cols = res.model.exog_names  # constant + feature order used in training

    def predict_with_beta(X_mat, beta):
        # X_mat is a NumPy array with columns in 'design_cols' order
        z = X_mat @ beta
        return 1.0 / (1.0 + np.exp(-z))

    def mc_probs_with_param_uncertainty(path_df, sims=500, seed=42):
        """
        path_df: DataFrame with base FEATURES (no constant). Rows = future dates (or any scenario path).
        Returns: DataFrame with p10, p50, p90 indexed by path_df.index.
        """
        # Build X with constant, then align to training design (fills missing with 0 if any)
        X = sm.add_constant(path_df[FEATURES], has_constant="add")
        X = X.reindex(columns=design_cols, fill_value=0.0)
        X_mat = X.to_numpy(dtype=float)

        rng = np.random.default_rng(seed)
        P = np.empty((sims, len(path_df)), dtype=float)

        for s in range(sims):
            beta_s = rng.multivariate_normal(beta_hat, cov_hat)
            P[s, :] = predict_with_beta(X_mat, beta_s)

        q10, q50, q90 = np.quantile(P, [0.10, 0.50, 0.90], axis=0)
        return pd.DataFrame({"p10": q10, "p50": q50, "p90": q90}, index=path_df.index)
else:
    print("Parameter-uncertainty MC skipped: final model is not simple_logit.")


In [ ]:
#Appendix, after scenarios exists
if best_name == "simple_logit":
    seeds = {"baseline": 221, "easing": 222, "stress": 223}
    mc_param = {name: mc_probs_with_param_uncertainty(df_s[FEATURES], sims=500, seed=seeds[name])
                for name, df_s in scenarios.items()}

    # Optional: save one combined CSV
    out_param = (pd.concat(
        [v.assign(scenario=k) for k, v in mc_param.items()]
    ).reset_index().rename(columns={"index":"Date"}))
    out_param.to_csv("paper3_scenario_probs_param_mc.csv", index=False)
    print("Saved parameter-uncertainty bands → paper3_scenario_probs_param_mc.csv")


Saved parameter-uncertainty bands → paper3_scenario_probs_param_mc.csv


In [ ]:
# Appendix: Parameter-uncertainty bands (statsmodels) — only if simple_logit won
if best_name == "simple_logit":
    param_bands = {}
    for name in ["baseline", "easing", "stress"]:
        path = scenarios[name][FEATURES]  # only the 4 linear features
        param_bands[name] = mc_probs_with_param_uncertainty(path, sims=1000, seed=123)

    plt.figure(figsize=(11,5))
    for name, q in param_bands.items():
        plt.fill_between(q.index, q["p10"], q["p90"], alpha=0.2,
                         label=f"{name.capitalize()} (param 10–90%)")
        plt.plot(q.index, q["p50"], lw=2)
    plt.ylim(0, 1)
    plt.ylabel("Recession Prob (12m ahead)")
    plt.title("Scenario Probabilities — Parameter Uncertainty Bands (statsmodels)")
    plt.legend(ncol=3, loc="upper center")
    plt.tight_layout()
    plt.show()

    # Optional: save a combined CSV
    out_param = (pd.concat([v.assign(scenario=k) for k, v in param_bands.items()])
                   .reset_index().rename(columns={"index": "Date"}))
    out_param.to_csv("paper3_scenario_probs_param_mc.csv", index=False)
    print("Saved parameter-uncertainty bands → paper3_scenario_probs_param_mc.csv")
else:
    print("Skipped parameter-uncertainty bands: final model is not simple_logit.")



Saved parameter-uncertainty bands → paper3_scenario_probs_param_mc.csv


In [ ]:
# Appendix: Scenario probability plot (from canonical CSV)
import pandas as pd
import matplotlib.pyplot as plt

#scen_wide = pd.read_csv("paper3_scenario_probs.csv", parse_dates=["Date"]).set_index("Date")
# [redacted] Critical operation disabled in public copy (data access or training).

plt.figure(figsize=(10,4))
for name in ["baseline", "easing", "stress"]:
    if name in scen_wide.columns:
        plt.plot(scen_wide.index, scen_wide[name], label=name.capitalize())
plt.ylim(0, 1)
plt.xlabel("Date"); plt.ylabel("Recession Prob (12m ahead)")
plt.title("Scenario-based Conditional Recession Probabilities (Final Model)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Appendix (Optional) confusion matrix at Youden or custom threshold
from sklearn.metrics import roc_curve, confusion_matrix, precision_score, recall_score, f1_score

fpr, tpr, thr = roc_curve(yte, p_hat_reg)
best_thr_l2 = thr[(tpr - fpr).argmax()]
y_pred = (p_hat_reg >= best_thr_l2).astype(int)

cm = confusion_matrix(yte, y_pred, labels=[0,1])
tn, fp, fn, tp = cm.ravel()
print(f"Youden thr (L2)={best_thr_l2:.3f} | TP={tp} FP={fp} FN={fn} TN={tn}")
print("Precision:", precision_score(yte, y_pred, zero_division=0),
      "Recall:", recall_score(yte, y_pred, zero_division=0),
      "F1:", f1_score(yte, y_pred, zero_division=0))

Youden thr (L2)=0.083 | TP=2 FP=28 FN=0 TN=57
Precision: 0.06666666666666667 Recall: 1.0 F1: 0.125


In [ ]:
# === Post-hoc performance summary (robust) ===
# Compares Yield-Curve-Only vs Macro-Augmented on TEST set
# Assumes:
#   - Split frames exist: train_df, test_df
#   - TARGET column is 'recession_t12'
#   - FEATURES (macro-augmented) includes 'YC_10y_3m' + 3 macro vars used in your Paper 3
import numpy as np, pandas as pd
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score

# --- config (edit only if your names differ) ---
TARGET = "recession_t12"
YIELD_ONLY = ["YC_10y_3m"]          # <- your YC feature name

# --- guards ---
for nm in ["train_df","test_df"]:
    if nm not in globals():
        raise RuntimeError(f"{nm} not found. Re-run the split cell that creates train/test frames.")
if "FEATURES" not in globals():
    raise RuntimeError("FEATURES not found. Re-run the cell that sets your 4 predictors.")

missing_train = sorted(set([TARGET] + YIELD_ONLY + FEATURES) - set(train_df.columns))
missing_test  = sorted(set([TARGET] + YIELD_ONLY + FEATURES) - set(test_df.columns))
if missing_train or missing_test:
    raise RuntimeError(f"Missing column[…redacted-path…]

def _fit_logit(df, cols):
    X = sm.add_constant(df[cols], has_constant="add").copy()
    y = df[TARGET].astype(int)
    # align/drop NA rows consistently
    X = X.replace([np.inf,-np.inf], np.nan).dropna(axis=0, how="any")
    y = y.loc[X.index]
    model = sm.Logit(y, X)
#    res = model.fit(disp=False)
# [redacted] Critical operation disabled in public copy (data access or training).
    return res, X.columns  # keep training design

def _predict_probs(res, df, train_cols):
    X = sm.add_constant(df[train_cols.difference(["const","_cons"], sort=False)], has_constant="add")
    # Reindex to exact train design, fill missing with 0 (none if same columns)
    X = X.reindex(columns=train_cols, fill_value=0.0)
    X = X.replace([np.inf,-np.inf], np.nan).dropna(axis=0, how="any")
    p = res.predict(X)
    return np.asarray(p), X.index

def _safe_auc(y_true, y_prob):
    try:
        return roc_auc_score(y_true, y_prob)
    except Exception:
        return np.nan

# --- Macro-augmented model ---
# Use in-memory 'res' if it exists; otherwise refit with FEATURES on train_df
if "res" in globals():
    res_macro = res
    # infer training column order
    macro_cols = pd.Index(getattr(res_macro, "params", pd.Series(index=[])).index)
    if macro_cols.empty:
        macro_cols = pd.Index(getattr(getattr(res_macro, "model", None), "exog_names", []))
else:
    res_macro, macro_cols = _fit_logit(train_df, FEATURES)

# --- Yield-curve-only baseline (refit quickly) ---
res_yc, yc_cols = _fit_logit(train_df, YIELD_ONLY)

# --- Evaluate on TEST set ---
y_test = test_df[TARGET].astype(int)

p_macro, idx_macro = _predict_probs(res_macro, test_df, macro_cols)
auc_macro = _safe_auc(y_test.loc[idx_macro], p_macro)

p_yc, idx_yc = _predict_probs(res_yc, test_df, yc_cols)
auc_yc = _safe_auc(y_test.loc[idx_yc], p_yc)

# --- McFadden pseudo-R^2 (training-set based; standard definition) ---
def mcfadden_r2(res_obj):
    try:
        return 1.0 - (res_obj.llf / res_obj.llnull)
    except Exception:
        return np.nan

r2_macro = mcfadden_r2(res_macro)
r2_yc    = mcfadden_r2(res_yc)

# --- assemble tidy summary ---
perf_summary = pd.DataFrame(
    [
        ["YieldCurveOnly",  auc_yc,    r2_yc],
        ["MacroAugmented",  auc_macro, r2_macro],
    ],
    columns=["Model","AUC (test)","McFadden R²"]
)
base_auc = perf_summary.loc[perf_summary["Model"]=="YieldCurveOnly","AUC (test)"].values[0]
base_r2  = perf_summary.loc[perf_summary["Model"]=="YieldCurveOnly","McFadden R²"].values[0]
perf_summary["ΔAUC vs YieldCurveOnly"] = perf_summary["AUC (test)"] - base_auc
perf_summary["ΔR² vs YieldCurveOnly"]  = perf_summary["McFadden R²"] - base_r2

print("Performance comparison (Paper 3):")
display(perf_summary.round(3))


Performance comparison (Paper 3):


Model  AUC (test)  McFadden R²  ΔAUC vs YieldCurveOnly  \
0  YieldCurveOnly       0.712        0.213                   0.000   
1  MacroAugmented       0.694        0.251                  -0.018   

   ΔR² vs YieldCurveOnly  
0                  0.000  
1                  0.038